In [1]:
import cv2
import os
import re
import numpy as np

In [2]:
tmpDir='/tmp'
tmpGeneratedFiles = []

In [3]:
flatSky = ['/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0741.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0730.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0745.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0733.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0736.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0738.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0739.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0737.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0746.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0732.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0735.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0748.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0731.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0734.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0743.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0749.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0742.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0744.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0740.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0747.CR2']
offset1600ISO = ['/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0992.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_1010.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_1020.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0991.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_1013.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_1005.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0998.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_1037.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_1006.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_1028.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_1012.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_1014.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_1009.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_1003.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_1026.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_1024.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_1032.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_1021.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_1015.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_1034.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_1029.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_1008.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_1004.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0990.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_1033.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_1001.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0999.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_1039.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0997.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_1007.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_1018.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_1019.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_1022.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_1011.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_1027.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_1002.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_1023.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_1036.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_1017.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_1025.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_1031.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_1038.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0993.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_1000.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0996.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_1016.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0995.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_1030.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_1035.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_1040.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0994.CR2']
dark60s1600ISO = ['/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0884.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0888.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0875.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0876.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0881.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0880.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0878.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0882.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0874.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0885.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0889.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0879.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0883.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0890.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0886.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0877.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0887.CR2']
ngc7000Images = ['/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0856.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0855.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0854.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0853.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0858.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0850.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0849.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0859.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0851.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0860.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0863.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0844.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0852.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0862.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0868.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0872.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0871.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0870.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0864.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0873.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0865.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0848.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0847.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0857.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0845.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0867.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0869.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0846.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0861.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0843.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0866.CR2']
alignedImages = ['/mnt/data/Photos/2018/08-Astro/to_be_sorted/Aligned_0019.tif', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/Aligned_0029.tif', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/Aligned_0008.tif', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/Aligned_0026.tif', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/Aligned_0006.tif', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/Aligned_0017.tif', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/Aligned_0028.tif', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/Aligned_0018.tif', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/Aligned_0016.tif', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/Aligned_0015.tif', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/Aligned_0024.tif', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/Aligned_0007.tif', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/Aligned_0009.tif', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/Aligned_0021.tif', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/Aligned_0003.tif', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/Aligned_0027.tif', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/Aligned_0011.tif', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/Aligned_0022.tif', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/Aligned_0025.tif', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/Aligned_0030.tif', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/Aligned_0001.tif', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/Aligned_0010.tif', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/Aligned_0013.tif', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/Aligned_0020.tif', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/Aligned_0005.tif', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/Aligned_0004.tif', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/Aligned_0012.tif', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/Aligned_0014.tif', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/Aligned_0002.tif', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/Aligned_0023.tif', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/Aligned_0000.tif']

In [4]:
#offsetList = ['/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0992.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_1010.CR2', '/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_1020.CR2']
offsetList = offset1600ISO
darkList = dark60s1600ISO
flatList = flatSky
imageList = ngc7000Images

In [5]:
def getFileName(fileName, removeExtention=False, removePath=True):
    '''
    get the name of a file from a string (can remove the extention or the complete path)
    
    parameters:
    fileName as a string. The name can include the path
    removeExtention as boolean. Default is False. This parameter remove the extention of the filename. (e.g: 'test.jpg'
    will become 'test')
    removePath as boolean. Default is True. If the complete path is given in the filename is it possible to remove it or not
    
    return value:
    the filename
    
    '''
    if removePath:
        fileName = os.path.basename(fileName)
    if removeExtention:
        re_catchFileName = re.compile('(.*)\..*$')
        catchFileNameResult = re_catchFileName.match(fileName)
        if catchFileNameResult:
            fileName = catchFileNameResult.group(1)

    return fileName

In [6]:
getFileName('/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0992.CR2', removeExtention=True, removePath=False)

'/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0992'

In [7]:
def uncompressRawFiles(rawFileList):
    '''
    Generate a pgm file from a raw file.
    
    Parameters:
    rawFileList as a list of strings. this parameter is the list of some filenames of camera RAW files.
    
    return value:
    a list of the generated files (with their path)
    '''
    generatedFiles = []
    for file in rawFileList:
        # Extract Raw data in a PGM file with dcraw
        print('dcraw -d -4 ' + file)
        os.system('dcraw -d -4 ' + file)
        generatedFiles.append(getFileName(file, removeExtention=True, removePath=False) + '.pgm')

    #print(generatedFiles)
    return generatedFiles     

In [8]:
def correctImageOrientation(image):
    if image.shape[0] < image.shape[1]:
        return image.T
    else:
        return image

In [9]:
def getRAWdata(rawFileList):
    '''
    Get the image raw data from a list of pgm files
    
    Parameters:
    rawFileList as a list of string. each element of the list is a string of the filname of a pgm file containing the raw data.
    
    Return Value:
    an array of array containing all images RAW data. Each element is an image.
    '''
    fileList = rawFileList[:]
    image = correctImageOrientation(cv2.imread(fileList.pop(0), flags=cv2.IMREAD_GRAYSCALE | cv2.IMREAD_IGNORE_ORIENTATION | cv2.IMREAD_ANYDEPTH))
    stackList = [image]

    for file in fileList:
        image = correctImageOrientation(cv2.imread(file, flags=cv2.IMREAD_GRAYSCALE | cv2.IMREAD_IGNORE_ORIENTATION | cv2.IMREAD_ANYDEPTH))
        stackList.append(image)
    
    stack = np.stack(stackList)
    
    return stack

In [10]:
def calculateMedian(RAWdataList):
    '''
    Calculate the median of a stack of RAW images
    
    Parameters:
    RAWdataList as an array of array of all images.
    
    Return value:
    The median of each pixel of the image stack
    '''
        
    #offsetStack = np.stack(offsetStackList)
    masterOffset = np.median(RAWdataList, axis=0)
    
    return masterOffset

# Generating master offset

The generation of the master offset is just a median of all offset image

In [11]:
RAWoffsetFiles = uncompressRawFiles(offsetList)
RAWoffsetData = getRAWdata(RAWoffsetFiles)
masterOffset = calculateMedian(RAWoffsetData)

dcraw -d -4 /mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0992.CR2
dcraw -d -4 /mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_1010.CR2
dcraw -d -4 /mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_1020.CR2
dcraw -d -4 /mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0991.CR2
dcraw -d -4 /mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_1013.CR2
dcraw -d -4 /mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_1005.CR2
dcraw -d -4 /mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0998.CR2
dcraw -d -4 /mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_1037.CR2
dcraw -d -4 /mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_1006.CR2
dcraw -d -4 /mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_1028.CR2
dcraw -d -4 /mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_1012.CR2
dcraw -d -4 /mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_1014.CR2
dcraw -d -4 /mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_1009.CR2
dcraw -d -4 /mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_1003.CR2
dcraw -d -4 /mnt/data/Photos/2018/

In [12]:
RAWoffsetData

array([[[160,  21,   0, ...,   0,   0,   0],
        [  0, 150,  64, ...,  75,   0,  75],
        [134,   0,   0, ...,   0, 225, 174],
        ..., 
        [ 10,   0,  32, ...,   0,   0,   0],
        [  0,  26, 375, ...,   0,  75, 160],
        [  0,  82, 155, ...,   0,  45,   0]],

       [[  0, 210,  85, ...,   0,   0,   0],
        [  0,   0,   0, ...,   0,  69,   0],
        [  0,   0,  80, ...,   0,  75,  96],
        ..., 
        [  0,   0,   0, ..., 120,   0, 160],
        [182, 135,   0, ...,  16, 112,  26],
        [  0,   0,   0, ...,  80,   0, 214]],

       [[ 96,   0,  10, ...,   0, 105,  16],
        [174,   0,   0, ...,   0,   0,   0],
        [  0,   0,  58, ...,   0, 187,   0],
        ..., 
        [335,  26,   0, ...,   0, 257,   0],
        [  0,  37,  75, ...,   0,   0,   0],
        [241,  37,   0, ...,   0,  58,   0]],

       ..., 
       [[  5, 187,  96, ...,  16, 150,   0],
        [  0,  69, 308, ..., 174,  26, 120],
        [ 64,   0,  32, ...,   0,  45, 

In [13]:
for i in range(len(RAWoffsetData)):
    print(RAWoffsetData[i].shape)

(5202, 3465)
(5202, 3465)
(5202, 3465)
(5202, 3465)
(5202, 3465)
(5202, 3465)
(5202, 3465)
(5202, 3465)
(5202, 3465)
(5202, 3465)
(5202, 3465)
(5202, 3465)
(5202, 3465)
(5202, 3465)
(5202, 3465)
(5202, 3465)
(5202, 3465)
(5202, 3465)
(5202, 3465)
(5202, 3465)
(5202, 3465)
(5202, 3465)
(5202, 3465)
(5202, 3465)
(5202, 3465)
(5202, 3465)
(5202, 3465)
(5202, 3465)
(5202, 3465)
(5202, 3465)
(5202, 3465)
(5202, 3465)
(5202, 3465)
(5202, 3465)
(5202, 3465)
(5202, 3465)
(5202, 3465)
(5202, 3465)
(5202, 3465)
(5202, 3465)
(5202, 3465)
(5202, 3465)
(5202, 3465)
(5202, 3465)
(5202, 3465)
(5202, 3465)
(5202, 3465)
(5202, 3465)
(5202, 3465)
(5202, 3465)
(5202, 3465)


In [14]:
masterOffset

array([[  10.,  105.,   53., ...,    0.,   67.,    0.],
       [   0.,   37.,    0., ...,   16.,   21.,   42.],
       [   0.,   75.,    0., ...,    0.,  105.,   85.],
       ..., 
       [   0.,    0.,    0., ...,    0.,   97.,    0.],
       [   0.,   67.,    0., ...,   16.,    0.,   37.],
       [ 112.,   42.,   67., ...,    0.,    5.,    0.]])

In [15]:
cv2.imwrite("masterOffset.pgm", masterOffset)

True

# Generating master Dark

In [16]:
RAWdarkFiles = uncompressRawFiles(darkList)
RAWdarkData = getRAWdata(RAWdarkFiles)
masterDark = calculateMedian(RAWdarkData)

dcraw -d -4 /mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0884.CR2
dcraw -d -4 /mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0888.CR2
dcraw -d -4 /mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0875.CR2
dcraw -d -4 /mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0876.CR2
dcraw -d -4 /mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0881.CR2
dcraw -d -4 /mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0880.CR2
dcraw -d -4 /mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0878.CR2
dcraw -d -4 /mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0882.CR2
dcraw -d -4 /mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0874.CR2
dcraw -d -4 /mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0885.CR2
dcraw -d -4 /mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0889.CR2
dcraw -d -4 /mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0879.CR2
dcraw -d -4 /mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0883.CR2
dcraw -d -4 /mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0890.CR2
dcraw -d -4 /mnt/data/Photos/2018/

In [17]:
masterDark

array([[  831.,   209.,  1663., ...,     0.,     0.,   590.],
       [    0.,     0.,     0., ...,     0.,    52.,    53.],
       [    0.,     0.,   308., ...,     0.,     0.,     0.],
       ..., 
       [   69.,    82.,     0., ...,     0.,   172.,    10.],
       [    0.,     0.,     0., ...,     0.,     0.,     0.],
       [    0.,    75.,  1094., ...,    48.,     0.,    53.]])

In [18]:
cv2.imwrite("masterDark.pgm", masterDark)

True

# Generating master Flat

In [19]:
RAWflatFiles = uncompressRawFiles(flatList)
RAWflatData = getRAWdata(RAWflatFiles)
RAWflatData = RAWflatData.astype(np.float64, copy=False)
RAWflatData -= masterOffset
masterFlat = calculateMedian(RAWflatData)
meanFlat = np.mean(masterFlat)

dcraw -d -4 /mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0741.CR2
dcraw -d -4 /mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0730.CR2
dcraw -d -4 /mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0745.CR2
dcraw -d -4 /mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0733.CR2
dcraw -d -4 /mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0736.CR2
dcraw -d -4 /mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0738.CR2
dcraw -d -4 /mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0739.CR2
dcraw -d -4 /mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0737.CR2
dcraw -d -4 /mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0746.CR2
dcraw -d -4 /mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0732.CR2
dcraw -d -4 /mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0735.CR2
dcraw -d -4 /mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0748.CR2
dcraw -d -4 /mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0731.CR2
dcraw -d -4 /mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0734.CR2
dcraw -d -4 /mnt/data/Photos/2018/

In [20]:
masterFlat

array([[ 21875. ,  14503. ,  20982.5, ...,  25216. ,  17814.5,  25605.5],
       [ 14461. ,  21687. ,  14318.5, ...,  17780.5,  25972. ,  17068.5],
       [ 20384. ,  14563. ,  19238.5, ...,  24143.5,  17314. ,  25029.5],
       ..., 
       [ 15867. ,  23187. ,  15213.5, ...,  19811.5,  27965.5,  19407.5],
       [ 21342.5,  15853.5,  22669.5, ...,  29957.5,  20932.5,  31518. ],
       [ 15444.5,  23118. ,  15706.5, ...,  20538.5,  29143.5,  19932.5]])

In [21]:
meanFlat

21864.837575319292

In [22]:
cv2.imwrite("masterFlat.pgm", masterFlat)

True

# Calculate images

In [23]:
# image= (Brute - black)/(PLU - offset)*meanFlat
#RAWimageFiles = uncompressRawFiles(imageList)
#RAWimageData = getRAWdata(RAWimageFiles)
#RAWimageData = RAWimageData.astype(np.float64, copy=False)
#images = ((RAWimageData - masterDark)/(masterFlat - masterOffser))*meanFlat

for i in imageList:
    RAWimageFiles = uncompressRawFiles([i])
    RAWimageData = getRAWdata(RAWimageFiles)
    #RAWimageData = RAWimageData.astype(np.float64, copy=False)
    image = ((RAWimageData - masterDark)/(masterFlat - masterOffset))*meanFlat
    #image = (RAWimageData - masterDark)
    
    index = imageList.index(i)
    # Debayreizing the RAW image
    image = image/image.max()*65535
    image = image.astype(np.uint16, copy=False)
    RGB_image = cv2.cvtColor(image[0], cv2.COLOR_BayerBG2RGB)
    # Gamma correction
    bitDepth = 2**16
    RGB_image = np.power(RGB_image/bitDepth, 1/2.1)*bitDepth
    RGB_image = RGB_image/RGB_image.max()*65535
    RGB_image = RGB_image.astype(np.uint16, copy=False)
    #RGB_image = cv2.cvtColor(RGB_image, cv2.COLOR_BGR2RGB)
    tgtFileName = getFileName(i, removeExtention=True, removePath=False) + '.png'
    print(tgtFileName)
    cv2.imwrite(tgtFileName, RGB_image)

dcraw -d -4 /mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0856.CR2
/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0856.png
dcraw -d -4 /mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0855.CR2
/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0855.png
dcraw -d -4 /mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0854.CR2
/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0854.png
dcraw -d -4 /mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0853.CR2
/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0853.png
dcraw -d -4 /mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0858.CR2
/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0858.png
dcraw -d -4 /mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0850.CR2
/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0850.png
dcraw -d -4 /mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0849.CR2
/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0849.png
dcraw -d -4 /mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0859.CR2
/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0

In [24]:
RAWimageFiles = uncompressRawFiles(['/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0856.CR2'])
RAWimageData = getRAWdata(RAWimageFiles)

dcraw -d -4 /mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0856.CR2


In [25]:
image = ((RAWimageData - masterDark)/(masterFlat - masterOffset))*meanFlat

In [26]:
image

array([[[ 12433.90763373,   5603.64291241,   7440.2815744 , ...,
           4873.11180097,   2953.09286762,   4902.3074152 ],
        [  5741.01295024,   3143.89096406,   4889.56314671, ...,
           3696.14158792,   2489.71504123,   4477.88380613],
        [ 10042.12173176,   5641.27297464,   7927.18985825, ...,
           4822.42674379,   3687.12642475,   5254.85382606],
        ..., 
        [  2047.71844942,   1503.10739186,   3337.17769415, ...,
           2036.22266494,   1127.4295924 ,   3154.53021826],
        [  4795.56306384,   2102.48145183,   2988.03532536, ...,
           5132.21042631,   2510.0300821 ,   4015.14011699],
        [  3326.96338257,   1823.96482633,   3464.37338225, ...,
           3729.21713009,   1415.95993289,   3837.11033932]]])

In [27]:
image = image / image.max()

In [28]:
image = image.astype(np.uint16, copy=False)

In [29]:
image[0]

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint16)

In [30]:
RGB_image = cv2.cvtColor(image[0], cv2.COLOR_BayerBG2RGB)

In [31]:
RGB_image = np.power(RGB_image/bitDepth, 1/2.1)*bitDepth

In [32]:
RGB_image

array([[[ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.]],

       [[ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.]],

       [[ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.]],

       ..., 
       [[ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.]],

       [[ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.]],

       [[ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,

In [33]:
RGB_image2 = RGB_image.astype(np.uint16, copy=False)
cv2.imwrite('test.png', RGB_image2)

True

In [34]:
cv2.imread('/mnt/data/Photos/2018/08-Astro/to_be_sorted/IMG_0856.png')

array([[[84, 65, 50],
        [84, 65, 50],
        [76, 62, 47],
        ..., 
        [61, 54, 45],
        [62, 54, 44],
        [62, 54, 44]],

       [[84, 65, 50],
        [84, 65, 50],
        [76, 62, 47],
        ..., 
        [61, 54, 45],
        [62, 54, 44],
        [62, 54, 44]],

       [[82, 66, 45],
        [82, 66, 45],
        [78, 65, 42],
        ..., 
        [61, 54, 43],
        [62, 54, 44],
        [62, 54, 44]],

       ..., 
       [[57, 44, 35],
        [57, 44, 35],
        [56, 51, 36],
        ..., 
        [60, 40, 36],
        [59, 45, 30],
        [59, 45, 30]],

       [[55, 41, 37],
        [55, 41, 37],
        [49, 48, 35],
        ..., 
        [63, 47, 35],
        [60, 45, 32],
        [60, 45, 32]],

       [[55, 41, 37],
        [55, 41, 37],
        [49, 48, 35],
        ..., 
        [63, 47, 35],
        [60, 45, 32],
        [60, 45, 32]]], dtype=uint8)

# Add aligned images

Hugin is used to align the generated images:

    align_image_stack -v --gpu -a Aligned_ *.png

In [40]:
image = cv2.imread(alignedImages.pop(0), cv2.IMREAD_IGNORE_ORIENTATION | cv2.IMREAD_ANYDEPTH | cv2.IMREAD_ANYCOLOR)
image = image.astype(np.float64, copy=False)

for i in alignedImages:
    print('Adding {}'.format(i))
    image += cv2.imread(i, cv2.IMREAD_IGNORE_ORIENTATION | cv2.IMREAD_ANYDEPTH | cv2.IMREAD_ANYCOLOR)



Adding /mnt/data/Photos/2018/08-Astro/to_be_sorted/Aligned_0008.tif
Adding /mnt/data/Photos/2018/08-Astro/to_be_sorted/Aligned_0026.tif
Adding /mnt/data/Photos/2018/08-Astro/to_be_sorted/Aligned_0006.tif
Adding /mnt/data/Photos/2018/08-Astro/to_be_sorted/Aligned_0017.tif
Adding /mnt/data/Photos/2018/08-Astro/to_be_sorted/Aligned_0028.tif
Adding /mnt/data/Photos/2018/08-Astro/to_be_sorted/Aligned_0018.tif
Adding /mnt/data/Photos/2018/08-Astro/to_be_sorted/Aligned_0016.tif
Adding /mnt/data/Photos/2018/08-Astro/to_be_sorted/Aligned_0015.tif
Adding /mnt/data/Photos/2018/08-Astro/to_be_sorted/Aligned_0024.tif
Adding /mnt/data/Photos/2018/08-Astro/to_be_sorted/Aligned_0007.tif
Adding /mnt/data/Photos/2018/08-Astro/to_be_sorted/Aligned_0009.tif
Adding /mnt/data/Photos/2018/08-Astro/to_be_sorted/Aligned_0021.tif
Adding /mnt/data/Photos/2018/08-Astro/to_be_sorted/Aligned_0003.tif
Adding /mnt/data/Photos/2018/08-Astro/to_be_sorted/Aligned_0027.tif
Adding /mnt/data/Photos/2018/08-Astro/to_be_sort

In [41]:
image

array([[[  41843.,   34292.,   23730.],
        [  41252.,   34512.,   23372.],
        [  40277.,   32347.,   22359.],
        ..., 
        [  36275.,   29416.,   22375.],
        [  37113.,   29889.,   23945.],
        [  37032.,   29836.,   23872.]],

       [[ 106788.,   85342.,   59130.],
        [ 105354.,   85274.,   58452.],
        [ 101004.,   82276.,   55062.],
        ..., 
        [  71293.,   58592.,   42404.],
        [  72329.,   59141.,   43633.],
        [  55120.,   44435.,   33161.]],

       [[ 176340.,  134819.,   95849.],
        [ 173874.,  134859.,   95000.],
        [ 164091.,  133044.,   90074.],
        ..., 
        [ 143757.,  116352.,   82721.],
        [ 127750.,  101220.,   74615.],
        [  72470.,   57116.,   42358.]],

       ..., 
       [[ 441438.,  363551.,  259014.],
        [ 440861.,  369121.,  261742.],
        [ 435502.,  367755.,  263181.],
        ..., 
        [ 419452.,  348242.,  252648.],
        [ 420891.,  349980.,  254084.],
     

In [42]:
image2 = image/image.max()*65535

In [45]:
lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB);

In [46]:
image2 = RGB_image.astype(np.uint16, copy=False)
cv2.imwrite('image.tif', image2)

True

In [39]:
# remove files: RAWoffsetFiles, RAWdarkFiles, RAWflatFiles and RAWimageFiles